In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys, os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# parallel processing
from multiprocessing import Pool
from functools import partial
Nprocs = 4

In [4]:
# samwich routines
samwich_path = os.path.join(os.environ['HOME'],'waketracking')
if not samwich_path in sys.path:
    sys.path.append(samwich_path)
from samwich.dataloaders import XarrayData
from samwich.waketrackers import track

In [5]:
# case definition
from study import V27, neutral

# Wake tracking with all methods

In [6]:
casedir = 'WAKEBENCH-NREL_DEBNATH_LES/neutral'
prefix = 'NREL_DEBNATH_LES_20190325'

In [7]:
case = neutral(casedir,prefix,turbine=V27)

In [8]:
Aref = case.turbine.rotor_area

## set up all tracking parameters

In [9]:
Gaussian2D_params = {
    'name': '2D Gaussian',
    'method': 'Gaussian2D',
    'umin': None, # use VD minima in each snapshot
    'A_ref': Aref,
    'A_min': Aref/5,
    'A_max': 2*Aref,
    'AR_max': 10.0,
}

In [ ]:
from samwich.gaussian_functions import Bastankhah
gauss = Bastankhah(CT=case.turbine.CTref, d0=case.turbine.D, TI=case.TI/100)

In [10]:
Bastankhah_params = {
    'name': '1D Gaussian (Bastankhah)',
    'method': 'Gaussian',
    'umin': { x_D: gauss.amplitude(x_D*case.turbine.D,-case.Uref) for x_D in case.downstreamD },
    'sigma': { x_D: gauss.sigma(x_D*case.turbine.D) for x_D in case.downstreamD },
}

Calculated k* : 0.0447339
Calculated Gaussian width : 8.598824208554174 m
Calculated Gaussian amplitude : -4.474464240770766 m/s
Calculated Gaussian width : 9.806639508554174 m
Calculated Gaussian amplitude : -3.112149184268989 m/s
Calculated Gaussian width : 11.014454808554174 m
Calculated Gaussian amplitude : -2.340643747870897 m/s
Calculated Gaussian width : 12.222270108554174 m
Calculated Gaussian amplitude : -1.8396952310704735 m/s
Calculated Gaussian width : 13.430085408554174 m
Calculated Gaussian amplitude : -1.4902034362444538 m/s
Calculated Gaussian width : 14.637900708554174 m
Calculated Gaussian amplitude : -1.2345920413753313 m/s
Calculated Gaussian width : 15.845716008554174 m
Calculated Gaussian amplitude : -1.0410934863247117 m/s
Calculated Gaussian width : 8.598824208554174 m
Calculated Gaussian width : 9.806639508554174 m
Calculated Gaussian width : 11.014454808554174 m
Calculated Gaussian width : 12.222270108554174 m
Calculated Gaussian width : 13.430085408554174 m
C

In [21]:
const_area_params = {
    'name': 'Constant-Area Contours',
    'method': 'ConstantArea',
    'ref_area': Aref,
}

## read all planes

In [23]:
plane3 = dataplanes[3]

In [36]:
np.concatenate(([1],np.array([2,3,4]),[5]))

array([1, 2, 3, 4, 5])

In [12]:
%%time
dataplanes = {
    downD: XarrayData(case.get_wake_datafile(downD),
                      uvar='U',vvar='V',wvar='W',
                      trim_time=case.trim_time(downD))
    for downD in case.downstreamD
}

Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_2D.nc
Calculated offset: 13
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_3D.nc
Calculated offset: 17
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_4D.nc
Calculated offset: 20
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_5D.nc
Calculated offset: 23
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_6D.nc
Calculated offset: 26
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_7D.nc
Calculated offset: 29
Selected datafile: WAKEBENCH-NREL_DEBNATH_LES/neutral/NREL_DEBNATH_LES_20190325_uvw_8D.nc
Calculated offset: 32
CPU times: user 7.13 s, sys: 8.02 s, total: 15.1 s
Wall time: 18.4 s


## run all trackers

In [13]:
def track_downstream_plane(downD,**tracking_params):
    method = tracking_params.pop('method')
    data = dataplanes[downD]
    # evaluate parameters that are a function of downstream distance
    for key,val in tracking_params.items():
        if isinstance(val, dict):
            tracking_params[key] = val[downD]
    print('x/D:',downD,':',tracking_params)
    # perform the wake tracking
    wo = track(data.sliceI(), method=method)
    wo.remove_shear(wind_profile=case.get_inflow(downD))
    yc,zc = wo.find_centers(**tracking_params, **case.get_outputs(method,downD))

### Gaussian 2D

In [14]:
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **Gaussian2D_params), case.downstreamD)
# CPU times: user 490 ms, sys: 203 ms, total: 693 ms
# Wall time: 45min 36s

x/D: 2 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
x/D: 5 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
x/D: 4 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
x/D: 3 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
Slicing data at i=0 x=135.0
Slicing data at i=0 x=54.0
Slicing data at i=0 x=108.0
Slicing data at i=0 x=81.0
Selected Tracker: Gaussian2D

Selected Tracker: Gaussian2D

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 81.0 0.0 40.5010165
Sampling plane normal vector: [1. 0. 0.]
Selected Tracker: Gaussian2D
Selected Tracker: Gaussian2D
  rotated to rotor-aligned axes (about z): 0.0 deg
  identified plane center at: 108.0 0.0 40.5010165


  rotation error: 0.0
  rotated 

Processed frame 1590


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/trajectory_2D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/outlines_2D.pkl
x/D: 6 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
Slicing data at i=0 x=162.0
Selected Tracker: Gaussian2D

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing Gaussian2D 

  subtracting out time-varying profile
average Gaussian function amplitude = -3.6066305061910957 m/s (over 1578 times)


Processed frame 1586


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/trajectory_3D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/outlines_3D.pkl
x/D: 7 : {'umin': None, 'A_ref': 572.5552611167398, 'A_min': 114.51105222334795, 'A_max': 1145.1105222334795, 'AR_max': 10.0}
Slicing data at i=0 x=189.0
Selected Tracker: Gaussian2D

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 189.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1575

...finished initializing WakeTracker

...finished initializing Gaussian2D 

  subtracting out time-varying profile
average Gaussian function amplitude = -3.389639083500555 m/s (over 1575 times)


...

Processed frame 1574


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/trajectory_7D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/outlines_7D.pkl


Processed frame 1571


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/trajectory_8D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian2D/outlines_8D.pkl
CPU times: user 490 ms, sys: 203 ms, total: 693 ms
Wall time: 45min 36s


[None, None, None, None, None, None, None]

### Gaussian 1D

In [15]:
# Bastankhah-Porte-Agel Gaussian
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **Bastankhah_params), case.downstreamD)
# CPU times: user 150 ms, sys: 79.8 ms, total: 230 ms
# Wall time: 7min 36s

x/D: 2 : {'umin': -4.474464240770766, 'sigma': 8.598824208554174}
x/D: 5 : {'umin': -1.8396952310704735, 'sigma': 12.222270108554174}
x/D: 3 : {'umin': -3.112149184268989, 'sigma': 9.806639508554174}
x/D: 4 : {'umin': -2.340643747870897, 'sigma': 11.014454808554174}
Slicing data at i=0 x=135.0
Slicing data at i=0 x=108.0
Slicing data at i=0 x=54.0
Slicing data at i=0 x=81.0
Selected Tracker: Gaussian
Selected Tracker: Gaussian
Selected Tracker: Gaussian
Selected Tracker: Gaussian




Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 108.0 0.0 40.5010165
  identified plane center at: 135.0 0.0 40.5010165
  identified plane center at: 54.0 0.0 40.5010165
  identified plane center at: 81.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 

Processed frame 1590


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/trajectory_2D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/outlines_2D.pkl
x/D: 6 : {'umin': -1.4902034362444538, 'sigma': 13.430085408554174}
Slicing data at i=0 x=162.0
Selected Tracker: Gaussian

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 162.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1578

...finished initializing WakeTracker

...finished initializing Gaussian 

  subtracting out time-varying profile
Average Gaussian function amplitude = -1.4902034362444534 m/s (over 1578 times)
Specified Gaussian width = 13.430085408554174 m
Reference Gaussian area = 566.6402518736617 m^2


Processed frame 1580


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/trajectory_5D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/outlines_5D.pkl
x/D: 7 : {'umin': -1.2345920413753313, 'sigma': 14.637900708554174}
Slicing data at i=0 x=189.0
Selected Tracker: Gaussian

Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 189.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  horizontal search range: -40.0 40.0
  vertical search range: 0.497933 80.5041
Number of time frames to process: 1575

...finished initializing WakeTracker

...finished initializing Gaussian 

  subtracting out time-varying profile
Average Gaussian function amplitude = -1.2345920413753309 m/s (over 1575 times)
Specified Gaussian width = 14.637900708554174 m
Reference Gaussian area = 673.1432055797768 m^2


...

Processed frame 1574


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/trajectory_7D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/outlines_7D.pkl


Processed frame 1571


Wrote out trajectory to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/trajectory_8D.csv
Wrote out pickled outlines to ./WAKEBENCH-NREL_DEBNATH_LES/neutral/Gaussian/outlines_8D.pkl
CPU times: user 150 ms, sys: 79.8 ms, total: 230 ms
Wall time: 7min 36s


[None, None, None, None, None, None, None]

### Constant area

In [17]:
p = Pool(Nprocs)
%time p.map(partial(track_downstream_plane, **const_area_params), case.downstreamD)
# CPU times: user 136 ms, sys: 32.7 ms, total: 169 ms
# Wall time: 4min 1s

x/D: 3 : {'ref_area': 572.5552611167398}
x/D: 2 : {'ref_area': 572.5552611167398}
x/D: 4 : {'ref_area': 572.5552611167398}
x/D: 5 : {'ref_area': 572.5552611167398}
Slicing data at i=0 x=108.0
Slicing data at i=0 x=135.0
Slicing data at i=0 x=81.0
Slicing data at i=0 x=54.0
Selected Tracker: ConstantArea
Selected Tracker: ConstantArea
Selected Tracker: ConstantArea
Selected Tracker: ConstantArea




Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
Sampling plane normal vector: [1. 0. 0.]
  identified plane center at: 81.0 0.0 40.5010165
  identified plane center at: 135.0 0.0 40.5010165
  identified plane center at: 54.0 0.0 40.5010165
  identified plane center at: 108.0 0.0 40.5010165
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotated to rotor-aligned axes (about z): 0.0 deg
  rotation error: 0.0
  rota

[None, None, None, None, None, None, None]

### Constant flux